# Evaluate RAG

In [1]:
import sys
sys.path.append("../..")
from datasets import load_dataset
from src.service.provider import ProviderService

provider = ProviderService()

# provider.config.enable_tracing(project="EVALUATE_RAG")

In [2]:
QA_REPO = "BroDeadlines/QA.UEH.QA_tu_van_tuyen_sinh"
QA_SPLIT = "train"
qa_dataset = load_dataset(QA_REPO, split=QA_SPLIT)
qa_dataset

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id'],
    num_rows: 54
})

# Run RAG

In [3]:
# donwload corpora dataset
# DATA_REPO = "BroDeadlines/TEST.UEH.ueh_copora_data"
DATA_REPO = "BroDeadlines/TEST.UEH.ueh_copora_data"
DATA_SPLIT = "train"

dataset = load_dataset(DATA_REPO)
dataset = dataset['train']
dataset

Dataset({
    features: ['url', 'content', 'doc_id', 'metadata', 'split', 'shards', 'propositions', 'proposition_list', '__index_level_0__'],
    num_rows: 196
})

In [4]:
from src.rag.hg_parent_retriever import HugFaceParentRAG
from src.rag.hyde_rag import HydeRAG, HydeHybridSearchRAG
from src.utils.type_utils import get_default_config

config = get_default_config()

# config['vec_index'] = "vec-sentence-propositon_medium_edu_tdt"
# config['txt_index'] = "text-sentence-propositon_medium_edu_tdt"

# config['vec_index'] = "vec-sentence-mini-hub-sentence"
# config['txt_index'] = "text-sentence-mines_sizei-hub-sentence"

# config['vec_index'] = "vec-sentence-compact-tdt-sentence"
# config['txt_index'] = "text-sentence-compact-tdt-sentence"
# config['vec_index'] = "vec-raptor-ueh-data-tree"
# config['txt_index'] = "text-raptor-ueh-data-tree"

# config['vec_index'] = "vec-sentence-ueh-sentence"
# config['txt_index'] = "text-sentence-ueh-sentence"

config['vec_index'] = "vec-sentence-ueh-unique"
config['txt_index'] = "text-sentence-ueh-unique"

config['total_k'] = 8
config['llm'] = "gemini-1.0-pro"
config

{'llm': 'gemini-1.0-pro',
 'total_k': 8,
 'txt_weight': 0.5,
 'vec_weight': 0.5,
 'vec_index': 'vec-sentence-ueh-unique',
 'txt_index': 'text-sentence-ueh-unique'}

In [7]:
UNI = "Đại học Kinh tế TP. Hồ Chí Minh"
rag = HugFaceParentRAG(provider=provider, config=config, text_corpora=dataset, uni = UNI)

# rag = HydeHybridSearchRAG(provider=provider, config=config)

In [6]:
# q = "sinh viên tôn đức thắng"
# q = "Đại học Tôn Đức Thắng có những phương thức tuyển sinh nào?"
# q = "Cho em hỏi là nếu em học chất lượg cao thì em sẽ học ở chi nhánh nào , sẽ học khác với đại trà chỗ nào và cơ sở vật chất ra làm sao ạ . Em cảm ơn"
q = "Câu 2: Học phí và học bổng đào tạo tại Vĩnh Long có gì khác so với đào tại Thành phố Hồ Chí Minh"
a = rag.answer(q)

a['answer']

'Học phí tại Vĩnh Long thấp hơn 65% so với Thành phố Hồ Chí Minh, học bổng cũng ít hơn về số lượng và giá trị.'

## Run

In [8]:
def eval_no_score1(row):
    resp = rag.answer(row['question'])
    row['answer'] = resp['answer']
    row['exc_second'] = resp['exc_second']
    docs = resp['retrieved_docs']
    result = [{'content': d.page_content, "doc_id": d.metadata['doc_id'], "score": -1.0} for d in docs]
    row['evaluation'] = result
    return row

In [9]:
eval_dataset = qa_dataset.map(eval_no_score1)

Parameter 'function'=<function eval_no_score1 at 0x7fb712ab83a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/54 [00:00<?, ? examples/s]

page_content='Bạn sẽ có rất nhiều câu hỏi để khám phá các cơ sở vật chất của UEH ngay từ những ngày đầu bỡ ngỡ.' metadata={'doc_id': '99c7f6be-168e-4a08-957c-fefe2094700a'}
page_content='Em chỉ cần thoả “Điều kiện đăng ký xét tuyển” theo từng Phương thức là em đã có thể nộp hồ sơ xét tuyển phương thức mình muốn.' metadata={'doc_id': '944c6354-fbfe-438b-b630-c2a0df97a69e'}
page_content='Điều kiện tham gia học bổng hỗ trợ học tập là sinh viên trúng tuyển vào UEH năm 2024.' metadata={'doc_id': '6937ab00-202f-4782-bed6-76551089fdb3'}
page_content='Năm 2024, UEH xét tuyển gồm 6 phương thức.' metadata={'doc_id': 'aa3ad130-fbd1-48bb-b7ca-a2a4ff6681e4'}
page_content='Đại học Kinh tế Thành phố Hồ Chí Minh thường đăng thông báo tuyển sinh 02 đợt vào tháng 01 và tháng 6 (tùy tình hình sẽ có thời gian chi tiết cụ thể về thời gian nộp hồ sơ dự tuyển, xét tuyển).' metadata={'doc_id': 'cc28dec2-a642-409d-8c85-efedeec297f4'}
page_content='Thời gian nộp hồ sơ nhập học kết thúc vào ngày 21/6/2024.' meta

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


page_content='Để đảm bảo kế hoạch học tập trong từng học kỳ, cũng như trong toàn khóa học, bạn cần chú ý:' metadata={'doc_id': '6f36549f-7719-4de2-a180-74ee410108ac'}
page_content='Trường hợp sinh viên không có tên trong danh sách nhận học bổng, sinh viên cần xem lại điểm rèn luyện của mình.' metadata={'doc_id': 'a94ae88b-2cdb-4248-bf58-64590325a806'}
page_content='Sinh viên học các chương trình Tiên tiến được ưu tiên bố trí học tại các cơ sở trung tâm của thành phố' metadata={'doc_id': '56961a22-5643-4ae6-8231-56f2319f8160'}
page_content='Em có thể nộp hồ sơ để xét học bổng hỗ trợ học tập.' metadata={'doc_id': 'f88bab16-aafe-4bce-8670-12a53fd80daa'}
page_content='Học bổng tạo cơ hội học tập cho các em có hoàn cảnh khó khăn.' metadata={'doc_id': 'e500cddd-2fa4-41ea-8971-300d89059838'}
page_content='Em không thể lấy lại điểm cũ.' metadata={'doc_id': '0ecaf7af-fbb5-4c82-9154-822fd2c21fda'}
page_content='Như vậy, em không thể lấy lại điểm cũ và phải đăng ký học lại học phần A.' metadata={

...

page_content='Phiếu thông tin sinh viên LTTCVLVH (In trong tài khoản đăng ký).' metadata={'doc_id': '6afd7804-64cb-4cce-ab34-e97fb451eb96'}
page_content='UEH đã triển khai các dịch vụ cung cấp hồ sơ điện tử có xác thực trực tiếp trên Portal sinh viên.' metadata={'doc_id': '5edd45ef-2c72-4d9f-874e-cc2fa7c515e7'}
page_content='Nhịp sống sinh viên là một phần của website UEH' metadata={'doc_id': '540c82e1-c8f4-48ad-a5f9-4c416140fd6c'}
page_content='Phiếu thông tin sinh viên LTCĐVLVH (In trong tài khoản đăng ký).' metadata={'doc_id': 'cbc2cfff-a2ba-4fc4-aa2d-d12d9290506a'}


Filter:   0%|          | 0/196 [00:00<?, ? examples/s]

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


page_content='Môi trường học tập mới như thế nào?' metadata={'doc_id': 'd0037893-a126-4d18-a984-a1de45e76f92'}
page_content='Mức cho vay tối đa của hộ gia đình là tổng số mức vay tối đa của từng sinh viên trong hộ.' metadata={'doc_id': '6dd99be2-878e-44ec-a0b0-e18d3fe33d49'}
page_content='Mức cho vay tối đa là 100% học phí HKC2023.' metadata={'doc_id': 'df237e88-7229-4870-8d37-43c52a9a4f33'}
page_content='Sinh viên cần phải trang bị những kỹ năng nào để phục vụ cho việc học tập?' metadata={'doc_id': 'd0037893-a126-4d18-a984-a1de45e76f92'}
page_content='Chương trình Cho vay học phí có thời hạn cho vay tối đa 3 tháng.' metadata={'doc_id': 'df237e88-7229-4870-8d37-43c52a9a4f33'}
page_content='Học đại học dễ hay khó?' metadata={'doc_id': 'd0037893-a126-4d18-a984-a1de45e76f92'}
page_content='Mức cho vay tối đa của Chương trình Cho vay học phí của Ngân hàng TMCP Phương Đông (OCB) là 10' metadata={'doc_id': 'df237e88-7229-4870-8d37-43c52a9a4f33'}
page_content='Mục tiêu của học tập bậc đại học

Retrying langchain_google_genai.llms._completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..


## Push

In [11]:
eval_dataset.save_to_disk("ueh_raptor")

Saving the dataset (0/1 shards):   0%|          | 0/54 [00:00<?, ? examples/s]

In [10]:
eval_dataset.push_to_hub("BroDeadlines/EVAL.RAG.UEH.evaluation", "clean")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/4.53k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/EVAL.RAG.UEH.evaluation/commit/0b9aea7977271f0cae56ab1424eea7c67e4d7a70', commit_message='Upload dataset', commit_description='', oid='0b9aea7977271f0cae56ab1424eea7c67e4d7a70', pr_url=None, pr_revision=None, pr_num=None)

## Update shards

### proposition

In [11]:
import json
def meta(row):
    row['metadata'] = json.dumps({"shards": 1})
    return row
eval_dataset = eval_dataset.map(meta)

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [12]:
test_ds = eval_dataset

### RAPTOR

In [15]:
CLUSTER_REPO = "BroDeadlines/TEST.PART_CLUSTER.UEH.raptor.edu_data"
SUMMERIZE_REPO = "BroDeadlines/TEST.PART_SUMMERIZE.UEH.raptor.edu_tdt_data"
SPLIT = "train"

cluster_dataset =  load_dataset(CLUSTER_REPO, split=SPLIT)
summerize_dataset =  load_dataset(SUMMERIZE_REPO, split=SPLIT)

In [20]:
easy_shards = {}
hard_shards = {}

def get_shards_cluster2(row):
    d_id = row['doc_ids']
    if d_id not in hard_shards:
        hard_shards[d_id] = row['hard_shards']
        easy_shards[d_id] = row['easy_shards']
    return row

# cluster_dataset.filter(lambda row: row['level_id'] == "tree_1").map(get_shards_cluster_1)
# summerize_dataset.map(get_shards_cluster_1)
len(easy_shards)

0

In [21]:
cluster_dataset.filter(lambda row: row['level_id'] == "tree_1").map(get_shards_cluster2)
len(easy_shards)

Map:   0%|          | 0/690 [00:00<?, ? examples/s]

255

In [22]:
summerize_dataset.map(get_shards_cluster2)
len(easy_shards)

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

440

In [24]:
import json
def update_shards(row):
    id = row['doc_id']
    return {**row, "metadata": json.dumps({"easy_shards": easy_shards[id], "hard_shards": hard_shards[id]})}

test_ds_update = eval_dataset.map(update_shards)
test_ds_update

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'url', 'group', 'doc_id', 'exc_second', 'evaluation', 'metadata'],
    num_rows: 54
})

In [27]:
test_ds_update.push_to_hub("BroDeadlines/EVAL.RAG.UEH.evaluation", "raptor")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.15k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/BroDeadlines/EVAL.RAG.UEH.evaluation/commit/d2ab09b895b68aa7aef2a6d31625ee02b7ca5dec', commit_message='Upload dataset', commit_description='', oid='d2ab09b895b68aa7aef2a6d31625ee02b7ca5dec', pr_url=None, pr_revision=None, pr_num=None)

# Report

In [3]:
EVAL_REPO = "BroDeadlines/EVAL.RAG.UEH.evaluation"
SUBSET = "raptor"
eval_dataset = load_dataset(EVAL_REPO, SUBSET)

Generating train split:   0%|          | 0/54 [00:00<?, ? examples/s]

In [4]:
eval_dataset = eval_dataset['train']

In [12]:
a = set([4,5])
len(a)

2

In [13]:
import numpy as np

count_docs = []
for ques in eval_dataset['evaluation']:
    docs = [q['doc_id'] for q in ques]
    doc_set = set(docs)
    count_docs.append(len(doc_set))

In [13]:
from src.utils.eval_utils import evaluate_IR, calculate_BLEU, calculate_ROUGE,evaluate_IR_RAPTOR

preds = eval_dataset['answer']
grounds = qa_dataset['answer']
len(preds) == len(grounds)

bleu = calculate_BLEU(grounds=grounds, preds=preds)
# bleu
rouge = calculate_ROUGE(grounds=grounds, preds=preds)

/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/h4438/miniconda3/envs/uni/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using low

### RAPTOR

In [14]:
data = {}
t_from = 6
t_to = 8

for i in range(t_from, t_to+1):
    res = evaluate_IR_RAPTOR(eval_dataset=eval_dataset, limit_k=i, shard_key='hard_shards')
    del res['relevant']
    data[f'k_{i}'] = res
data

KeyError: 'hard_shards'

### Normal

In [16]:
data = {}
t_from = 6
t_to = 8

for i in range(t_from, t_to+1):
    res = evaluate_IR(eval_dataset=eval_dataset, limit_k=i, shard_key='shards')
    del res['relevant']
    data[f'k_{i}'] = res
data

{'k_6': {'precision': 0.5425531914893617,
  'recall': 0.9444444444444444,
  'map_score': 0.2328703703703704,
  'relevant_retrieved': 51,
  'num_retrieved': 94},
 'k_7': {'precision': 0.5555555555555556,
  'recall': 1.0185185185185186,
  'map_score': 0.23323360208280844,
  'relevant_retrieved': 55,
  'num_retrieved': 99},
 'k_8': {'precision': 0.5544554455445545,
  'recall': 1.037037037037037,
  'map_score': 0.2319475938523558,
  'relevant_retrieved': 56,
  'num_retrieved': 101}}

In [17]:
import numpy as np
time = eval_dataset['exc_second']
time_np = np.array(time)

In [13]:
es = provider.get_elasticsearch_store(index=config['vec_index'])

NameError: name 'config' is not defined

In [14]:
es_connect = provider.config.load_elasticsearch_connection()

es_connect

FileNotFoundError: [Errno 2] No such file or directory: '/home/h4438/Desktop/graduate/WebQA/core/experiments/evaluation/../../src/service/../.keys/elastic.nodes'

In [18]:
import json

ALGO = ['proposition', 'parent retriever', 'hybrid search']
ES_SIZE = 883

a = {
    "QA": {"repo": QA_REPO, "split": QA_SPLIT,
           "size": qa_dataset.num_rows, 
           "total_time(minute)": round(time_np.sum() / 60, 3), 
           "median_time(second)": round(np.median(time_np), 3),
           "avg_time(second)": round(time_np.mean(), 3)},
    "RAG": {"algo": ALGO, **config, "es_size": ES_SIZE},
    "IR": data,
    "BLEU": bleu,
    "ROUGE-L": rouge
}
 
json_formatted_str = json.dumps(a, indent=2)
print(json_formatted_str)

{
  "QA": {
    "repo": "BroDeadlines/QA.UEH.QA_tu_van_tuyen_sinh",
    "split": "train",
    "size": 54,
    "total_time(minute)": 21.167,
    "median_time(second)": 23.754,
    "avg_time(second)": 23.519
  },
  "RAG": {
    "algo": [
      "proposition",
      "parent retriever",
      "hybrid search"
    ],
    "llm": "gemini-1.0-pro",
    "total_k": 8,
    "txt_weight": 0.5,
    "vec_weight": 0.5,
    "vec_index": "vec-sentence-ueh-unique",
    "txt_index": "text-sentence-ueh-unique",
    "es_size": 883
  },
  "IR": {
    "k_6": {
      "precision": 0.5425531914893617,
      "recall": 0.9444444444444444,
      "map_score": 0.2328703703703704,
      "relevant_retrieved": 51,
      "num_retrieved": 94
    },
    "k_7": {
      "precision": 0.5555555555555556,
      "recall": 1.0185185185185186,
      "map_score": 0.23323360208280844,
      "relevant_retrieved": 55,
      "num_retrieved": 99
    },
    "k_8": {
      "precision": 0.5544554455445545,
      "recall": 1.037037037037037,
